In [229]:
import pandas as pd

In [230]:
folder = '.\\input_files\\'

# file = '420680448_mercyone-des-moines-medical-center_standardcharges.xlsx'
file = '421470935_mercyone-newton-medical-center_standardcharges.csv'

In [231]:
file_path = folder + file
    
if file.endswith('.xlsx'):
    df = pd.read_excel(file_path, dtype=str, skiprows=4)

elif file == '421470935_mercyone-newton-medical-center_standardcharges.csv':
    df = pd.read_csv(file_path, dtype=str, skiprows=3)
elif file.endswith('.csv'):
    try:
        df = pd.read_csv(file_path, dtype=str, skiprows=4)
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, dtype=str, skiprows=4, encoding='ansi')

In [232]:
df.rename(columns=lambda x: x.strip(), inplace=True)
df.rename(columns={
    'Code': 'code',
    'Description': 'description',
    'Code Type': 'line_type',
    'Revenue Code (RC)': 'rev_code'
}, inplace=True)

In [233]:
df_copy = df.copy()

In [234]:
df = df_copy.copy()

In [235]:
df.drop(df.tail(9).index, inplace=True)

In [236]:
cols = df.columns.tolist()
id_vars = cols[:4]
value_vars = cols[4:]

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='standard_charge')

In [237]:
df

,code,description,line_type,rev_code,payer_name,standard_charge
0,1,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,MS-DRG,No RC,Gross Charge,*
1,2,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,MS-DRG,No RC,Gross Charge,*
2,3,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,MS-DRG,No RC,Gross Charge,*
3,4,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,MS-DRG,No RC,Gross Charge,*
4,5,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,MS-DRG,No RC,Gross Charge,*
...,...,...,...,...,...,...
269037,V5240,Disp fee contralateral binau,HCPCS,No RC,Medicare | Medigold | All Plans,NaN
269038,V5241,"Dispensing fee, monaural",HCPCS,No RC,Medicare | Medigold | All Plans,NaN
269039,V5362,Speech screening,HCPCS,No RC,Medicare | Medigold | All Plans,NaN
269040,V5363,Language screening,HCPCS,No RC,Medicare | Medigold | All Plans,NaN


In [238]:
df.loc[df['rev_code'] == 'No RC', 'rev_code'] = pd.NA
df.loc[(df['standard_charge'] == '*') | (df['standard_charge'] == 'IP') | (df['standard_charge'] == '**'), 'standard_charge'] = pd.NA

In [239]:
df.dropna(subset='standard_charge', inplace=True)

In [240]:
mask = (~df['standard_charge'].isna() & df['standard_charge'].str.contains('%'))
df.loc[mask, 'standard_charge_percent'] = df['standard_charge'].str.split('%').str[0]

In [241]:
df.loc[~df['standard_charge_percent'].isna(), ['standard_charge', 'contracting_method']] = (pd.NA, 'percent of total billed charge')

In [242]:
df.loc[df['line_type'].str.match(r'HCPCS|CPT'), 'hcpcs_cpt'] = df['code']
df.loc[df['line_type'] == 'MS-DRG', 'ms_drg'] = df['code'].str.zfill(3)

In [243]:
mask = df['line_type'] == 'APR-DRG'
df.loc[mask & (df['code'].str.len() <= 3), 'apr_drg'] = df['code'].str.zfill(3)

df.loc[mask & (df['code'].str.len() == 4), 'apr_drg'] = df['code'].str[:3] + '-' + df['code'].str[3:]


In [244]:
mapping = {
    'Gross Charge': 'gross',
    'Discounted Cash Price': 'cash',
    'De-Identified Minimum Negotiated Charge': 'min',
    'De-Identified Maximum Negotiated Charge': 'max'
}

df['payer_category'] = df['payer_name'].map(mapping).fillna('payer')

In [245]:
df['plan_name'] = pd.NA

In [246]:
df.loc[df['payer_name'].str.contains('|', regex=False), 'plan_name'] =  df['payer_name'].str.split('| ', regex=False).str[-1]

In [247]:
mapping = {'421470935_mercyone-newton-medical-center_standardcharges.csv': '160032',
 '420758901_MercyOne Cedar Falls Medical Center_standardcharges.xlsx': '160040',
 '311373080_Mercy Medical Center North Iowa_standardcharges.xlsx': '160064',
 '421264647_MercyOne Waterloo Medical Center_standardcharges.xlsx': '160067',
 '421437483_Mercy Medical Center Dubuque_standardcharges.xlsx': '160069',
 '421336618_Mercy Medical Center Clinton_standardcharges.xlsx': '160080',
 '420680448_mercyone-des-moines-medical-center_standardcharges.xlsx': '160083',
 '311407377_Mercy Medical Center Siouxland_standardcharges.xlsx': '160153',
 '421500277_MercyOne Primghar Medical Center_standardcharges.xlsx': '161300',
 '420818642_MercyOne Elkader Medical Center_standardcharges.xlsx': '161319',
 '421178403_MercyOne Oelwein Medical Center_standardcharges.xlsx': '161338',
 '420680308_mercyone-centerville-medical-center_standardcharges.csv': '161377',
 '202552602_Mercy Medical Center Dyersville_standardcharges.xlsx': '161378'}

hospital_id = mapping[file]
df['hospital_id'] = hospital_id

In [248]:
df.loc[df['ms_drg'].str.len() ]

,code,description,line_type,rev_code,payer_name,standard_charge,standard_charge_percent,contracting_method,hcpcs_cpt,ms_drg,apr_drg,payer_category,plan_name,hospital_id
34577,7892,"Level 2: Neonates, Died or Transferred to Anot...",MS-DRG,<NA>,De-Identified Minimum Negotiated Charge,4037.39,NaN,NaN,NaN,7892,NaN,min,NaN,160032
34580,7897,"Level 3: Neonates, Died or Transferred to Anot...",MS-DRG,<NA>,De-Identified Minimum Negotiated Charge,33763.24,NaN,NaN,NaN,7897,NaN,min,NaN,160032
34584,7902,Level 2: Extreme Immaturity or Respiratory Dis...,MS-DRG,<NA>,De-Identified Minimum Negotiated Charge,30729.61,NaN,NaN,NaN,7902,NaN,min,NaN,160032
34590,7907,Level 3: Extreme Immaturity or Respiratory Dis...,MS-DRG,<NA>,De-Identified Minimum Negotiated Charge,67988.42,NaN,NaN,NaN,7907,NaN,min,NaN,160032
34595,7912,Level 2: Prematurity W Major Problems,MS-DRG,<NA>,De-Identified Minimum Negotiated Charge,18077.98,NaN,NaN,NaN,7912,NaN,min,NaN,160032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177052,7937,Level 3: Full Term Neonate W Major Problems,MS-DRG,<NA>,Medicaid | Iowa Total Care | All Plans,11310.63,NaN,NaN,NaN,7937,NaN,payer,All Plans,160032
177055,7942,Level 2: Neonate W Other Significant Problems,MS-DRG,<NA>,Medicaid | Iowa Total Care | All Plans,2653.23,NaN,NaN,NaN,7942,NaN,payer,All Plans,160032
177058,7947,Level 3: Neonate W Other Significant Problems,MS-DRG,<NA>,Medicaid | Iowa Total Care | All Plans,2653.23,NaN,NaN,NaN,7947,NaN,payer,All Plans,160032
177062,7952,Level 2: Normal Newborn,MS-DRG,<NA>,Medicaid | Iowa Total Care | All Plans,2309.6,NaN,NaN,NaN,7952,NaN,payer,All Plans,160032


In [249]:
output_folder = '.\\output_files\\'
# df.to_csv(output_folder + hospital_id + "_" + file.split('_')[1].replace(' ', '_') + '.csv', index=False)